In [1]:
!pip install pyspark
!pip install eli5


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, avg
from pyspark.sql.types import DateType, FloatType, DoubleType, IntegerType
from pyspark.ml.feature import StringIndexer, Imputer, VectorAssembler, ChiSqSelector
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
import time


In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
df = spark.read.csv('flaskapp/data/hwc.csv', header=True, inferSchema=True)
df.show()

+--------------------+---------------+--------------------+------+----------+---------+----------------+----------------+------------+----------------+--------+------------------+------------------+--------------+---------+------------------+------------------+--------------+-----------------+---------------------------+---------------------------+-----------------------+--------------+------------------------+------------------------+--------------------+-------------+-----------------------+-----------------------+-------------------+-------+-----------------+-----------------+-------------+-------------------+---------+----------+--------+---------+----------+------------+-------------+-------+---------------+---------------+----------+--------------------+--------------------+-------------+-----------------------+-----------------------+-------------------+------+----------------+----------------+------------+--------+------------------+------------------+--------------+-----------

In [4]:
compact_data = df.drop('P_GEO_ALBEDO', 'P_DETECTION_MASS', 'P_DETECTION_RADIUS', 'P_ALT_NAMES', 'P_ATMOSPHERE', 'S_DISC', 'S_MAGNETIC_FIELD',
                 'P_TEMP_MEASURED', 'P_TPERI',
'P_DENSITY', 'P_ESCAPE', 'P_GRAVITY', 'P_POTENTIAL', 'P_OMEGA', 'P_INCLINATION', 'P_IMPACT_PARAMETER', 'P_HILL_SPHERE',
                 'P_MASS')

error_columns = [col for col in compact_data.columns if 'ERROR' in col]
df_cleaned = compact_data.drop(*error_columns)
df_cleaned.show()



+--------------------+---------------+--------------------+------+----------+------------+----------------+--------+--------------+---------+--------------+-----------------+-----------------------+--------------+--------------------+-------------------+-------------+-------------------+---------+----------+--------+---------+----------+------------+-------------+-------+----------+-------------+-------------------+------+------------+--------+--------------+-------------+-------------------+-----+-----------+---------+---------------+-------+-------------+-----------+------------+----------+--------------+------------+------------+------------+------------+----------------+----------------+-----------+---------------+---------------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-----------+-----------+------------+-------------+-------------+-----------+---------

In [5]:
df_cleaned.dtypes

[('P_NAME', 'string'),
 ('P_DETECTION', 'string'),
 ('P_DISCOVERY_FACILITY', 'string'),
 ('P_YEAR', 'int'),
 ('P_UPDATE', 'date'),
 ('P_MASS_LIMIT', 'int'),
 ('P_MASS_ORIGIN', 'string'),
 ('P_RADIUS', 'double'),
 ('P_RADIUS_LIMIT', 'int'),
 ('P_PERIOD', 'double'),
 ('P_PERIOD_LIMIT', 'int'),
 ('P_SEMI_MAJOR_AXIS', 'double'),
 ('P_SEMI_MAJOR_AXIS_LIMIT', 'int'),
 ('P_ECCENTRICITY', 'double'),
 ('P_ECCENTRICITY_LIMIT', 'int'),
 ('P_INCLINATION_LIMIT', 'int'),
 ('P_OMEGA_LIMIT', 'int'),
 ('S_NAME', 'string'),
 ('S_NAME_HD', 'string'),
 ('S_NAME_HIP', 'string'),
 ('S_TYPE', 'string'),
 ('S_RA', 'double'),
 ('S_DEC', 'double'),
 ('S_RA_STR', 'string'),
 ('S_DEC_STR', 'string'),
 ('S_MAG', 'double'),
 ('S_DISTANCE', 'double'),
 ('S_TEMPERATURE', 'double'),
 ('S_TEMPERATURE_LIMIT', 'int'),
 ('S_MASS', 'double'),
 ('S_MASS_LIMIT', 'int'),
 ('S_RADIUS', 'double'),
 ('S_RADIUS_LIMIT', 'int'),
 ('S_METALLICITY', 'double'),
 ('S_METALLICITY_LIMIT', 'int'),
 ('S_AGE', 'double'),
 ('S_AGE_LIMIT', 'i

In [6]:
df_cleaned.count()

5599

In [7]:
#calcul des valeurs null dans chaque colonne du dataFrame
null_counts = df_cleaned.select([count(when(col(c).isNull(), c)).alias(c) for c in df_cleaned.columns])


In [8]:
mean_values = {
    col_name: df_cleaned.select(avg(col_name)).first()[0]
    for col_name, dtype in df_cleaned.dtypes
    if dtype != 'string' and not isinstance(df_cleaned.schema[col_name].dataType, DateType)
}
compact_data = df_cleaned.na.fill(mean_values)


In [9]:
compact_data.show()

+--------------------+---------------+--------------------+------+----------+------------+----------------+--------+--------------+-----------------+--------------+-----------------+-----------------------+-------------------+--------------------+-------------------+-------------+-------------------+---------+----------+--------+---------+----------+------------+-------------+------------------+-----------------+----------------+-------------------+------+------------+-----------------+--------------+--------------------+-------------------+-----------------+-----------+--------------------+---------------+-----------------+-------------+-----------+------------+----------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+-----------+-----------+------------+-----------------+------------------+------------------+------------------+-----------------

# Convert Categorical Features to Numerical

In [10]:
# Identifier les colonnes catégorielles
categorical_columns = [col_name for col_name, dtype in compact_data.dtypes if dtype == 'string']

# Créer un indexeur pour chaque colonne catégorielle
indexers = [StringIndexer(inputCol=column, outputCol=column + "_indexed").setHandleInvalid("keep") for column in categorical_columns]

# Créer le pipeline
pipeline = Pipeline(stages=indexers)

# Appliquer le pipeline au DataFrame
compact_data = pipeline.fit(compact_data).transform(compact_data)

#supprimer les anciennes colonnes catégorielles si nécessaire
for column in categorical_columns:
    compact_data = compact_data.drop(column)



In [11]:
# Identifier les colonnes numériques pour l'imputation
numeric_cols = [col_name for col_name, dtype in compact_data.dtypes if dtype in ['int', 'double']]

# Créer un imputer pour les colonnes numériques identifiées
imputer = Imputer(
    inputCols=numeric_cols,
    outputCols=["{}_imputed".format(col_name) for col_name in numeric_cols]
).setStrategy("mean")  # ou "median" selon le cas d'usage

# Appliquer l'imputation
compact_data = imputer.fit(compact_data).transform(compact_data)

compact_data.show()

+------+----------+------------+--------+--------------+-----------------+--------------+-----------------+-----------------------+-------------------+--------------------+-------------------+-------------+---------+----------+------------------+-----------------+----------------+-------------------+------+------------+-----------------+--------------+--------------------+-------------------+-----------------+-----------+--------------------+---------------+-----------------+-------------+-----------+------------+----------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------+-------------+-------------+-----------+-------------------+----------

# Removing Multicollinearity

In [12]:
# Identifier les colonnes numériques
numeric_cols = [col_name for col_name, dtype in compact_data.dtypes if dtype in ['int', 'double']]

# Assembler les colonnes numériques dans un vecteur
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
feature_vector_df = assembler.transform(compact_data)

# Calculer la matrice de corrélation
correlation_matrix = Correlation.corr(feature_vector_df, "features").head()[0]

# Extraire la matrice de corrélation en tant que tableau Numpy (si nécessaire)
correlation_array = correlation_matrix.toArray()

# Afficher la matrice de corrélation
print(correlation_array)

[[ 1.          0.00902213 -0.21140022 ... -0.00558671 -0.00558671
  -0.00619957]
 [ 0.00902213  1.         -0.05395405 ... -0.01573223 -0.01573223
  -0.01567132]
 [-0.21140022 -0.05395405  1.         ...  0.44583817  0.44583817
   0.44617999]
 ...
 [-0.00558671 -0.01573223  0.44583817 ...  1.          1.
   0.99989613]
 [-0.00558671 -0.01573223  0.44583817 ...  1.          1.
   0.99989613]
 [-0.00619957 -0.01567132  0.44617999 ...  0.99989613  0.99989613
   1.        ]]


In [13]:
import numpy as np

# Seuil de corrélation haute pour retirer les colonnes
high_corr_threshold = 0.9
cols_to_remove = []

for i in range(len(correlation_array)):
    for j in range(i+1, len(correlation_array)):
        if abs(correlation_array[i, j]) > high_corr_threshold:
            cols_to_remove.append(numeric_cols[j])

# Supprimer les colonnes sélectionnées
cols_to_remove = list(set(cols_to_remove))  # Enlever les doublons
compact_data = compact_data.drop(*cols_to_remove)


# Removal of Outliers

In [14]:
# Calculer Q1 et Q3 pour chaque colonne numérique et en déduire l'IQR
#quantile_probs = [0.25, 0.75]
#rel_error = 0.05  # Erreur relative pour le calcul approximatif des quantiles

# Calculer les quantiles pour chaque colonne numérique
#Q1 = {col_name: compact_data.approxQuantile(col_name, [0.25], rel_error)[0] for col_name in compact_data.columns if compact_data.schema[col_name].dataType in [FloatType(), DoubleType(), IntegerType()]}
#Q3 = {col_name: compact_data.approxQuantile(col_name, [0.75], rel_error)[0] for col_name in compact_data.columns if compact_data.schema[col_name].dataType in [FloatType(), DoubleType(), IntegerType()]}

# Calculer l'IQR pour chaque colonne numérique
#IQR = {col_name: Q3[col_name] - Q1[col_name] for col_name in Q1}

# Filtrer les outliers
#for col_name in IQR:
#    lower_bound = Q1[col_name] - 1.5 * IQR[col_name]
#    upper_bound = Q3[col_name] + 1.5 * IQR[col_name]
#    compact_data = compact_data.filter((col(col_name) >= lower_bound) & (col(col_name) <= upper_bound))

# Final state of dataframe

In [15]:
compact_data.show()

+------+----------+------------+--------+--------------+-----------------+--------------+-----------------+-----------------------+-------------------+--------------------+-------------------+-------------+---------+----------+------------------+-----------------+----------------+-------------------+------+------------+-----------------+--------------+--------------------+-------------------+-----------------+-----------+--------------------+---------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+--------------------+------------+-------------+-------------+-----------+-------------------+--------------+-------------------+----------------------------+---------------------+--------------+-----------------+------------------+--------------+----------------+-----------------+--------------+-------------------+-------------------+-----------------------+---------------------+--------------------+----------------------+--

In [16]:
compact_data = compact_data.drop('P_UPDATE')

In [17]:
feature_columns = [col for col in compact_data.columns if col != 'P_HABITABLE']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data_vector = assembler.transform(compact_data)

In [18]:
selector = ChiSqSelector(numTopFeatures=15, featuresCol="features", outputCol="selectedFeatures", labelCol="P_HABITABLE")
selected_data = selector.fit(data_vector).transform(data_vector)

selected_data.show()

+------+------------+--------+--------------+-----------------+--------------+-----------------+-----------------------+-------------------+--------------------+-------------------+-------------+---------+----------+------------------+-----------------+----------------+-------------------+------+------------+-----------------+--------------+--------------------+-------------------+-----------------+-----------+--------------------+---------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+--------------------+------------+-------------+-------------+-----------+-------------------+--------------+-------------------+----------------------------+---------------------+--------------+-----------------+------------------+--------------+----------------+-----------------+--------------+-------------------+-------------------+-----------------------+---------------------+--------------------+----------------------+-------------

In [19]:
train_data, test_data = selected_data.randomSplit([0.8, 0.2], seed=42)

In [20]:
print("Distribution of 'P_HABITABLE':")
train_data.groupBy('P_HABITABLE').count().show()


Distribution of 'P_HABITABLE':
+-----------+-----+
|P_HABITABLE|count|
+-----------+-----+
|          1|   19|
|          2|   35|
|          0| 4476|
+-----------+-----+



In [21]:
# Configuration des modèles
rf = RandomForestClassifier(featuresCol='selectedFeatures', labelCol='P_HABITABLE', maxBins=300)
lr = LogisticRegression(featuresCol='selectedFeatures', labelCol='P_HABITABLE', maxIter=10, family="multinomial")
dt = DecisionTreeClassifier(featuresCol='selectedFeatures', labelCol='P_HABITABLE', maxBins=300)

# Entraînement des modèles
rf_model = rf.fit(train_data)
lr_model = lr.fit(train_data)
dt_model = dt.fit(train_data)

# Évaluation des modèles
evaluator = MulticlassClassificationEvaluator(labelCol='P_HABITABLE', metricName="accuracy")

print("Random Forest Accuracy:", evaluator.evaluate(rf_model.transform(test_data)))
print("Logistic Regression Accuracy:", evaluator.evaluate(lr_model.transform(test_data)))
print("Decision Tree Accuracy:", evaluator.evaluate(dt_model.transform(test_data)))

# Temps Random Forest
start_time_rf = time.time()
rf_model = rf.fit(train_data)
end_time_rf = time.time()

# Temps Decision Tree
start_time_dt = time.time()
dt_model = dt.fit(train_data)
end_time_dt = time.time()

# Affichage des temps d'exécution
print(f"Random Forest training time: {end_time_rf - start_time_rf} seconds")
print(f"Decision Tree training time: {end_time_dt - start_time_dt} seconds")



Random Forest Accuracy: 1.0
Logistic Regression Accuracy: 0.9971936389148737
Decision Tree Accuracy: 1.0
Random Forest training time: 1.407177209854126 seconds
Decision Tree training time: 1.0592198371887207 seconds


In [22]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import col

#  Random Forest
rf_predictions = rf_model.transform(test_data).select(col("prediction").cast("float"), col("P_HABITABLE").cast("float"))
rf_predictions_rdd = rf_predictions.rdd.map(tuple)
metrics_rf = MulticlassMetrics(rf_predictions_rdd)

print("Random Forest - Confusion Matrix:\n", metrics_rf.confusionMatrix().toArray())

# Logistic regression

lr_predictions = lr_model.transform(test_data).select(col("prediction").cast("float"), col("P_HABITABLE").cast("float"))
lr_predictions_rdd = lr_predictions.rdd.map(tuple)
metrics_lr = MulticlassMetrics(lr_predictions_rdd)

print("Logistic Regression - Confusion Matrix:\n", metrics_lr.confusionMatrix().toArray())

# Decision Tree
dt_predictions = dt_model.transform(test_data).select(col("prediction").cast("float"), col("P_HABITABLE").cast("float"))
dt_predictions_rdd = dt_predictions.rdd.map(tuple)
metrics_dt = MulticlassMetrics(dt_predictions_rdd)

print("Decision Tree - Confusion Matrix:\n", metrics_dt.confusionMatrix().toArray())


C:\Users\lucas\PycharmProjects\platenary-project\venv\Lib\site-packages\pyspark\sql\context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 341.0 failed 1 times, most recent failure: Lost task 0.0 in stage 341.0 (TID 434) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache

In [ ]:
def print_confusion_matrix(matrix, title="Confusion Matrix"):
    print(title)
    for row in matrix.toArray():
        print(' '.join(f"{int(value):5d}" for value in row))

# Affichage des matrices de confusion sans notation scientifique
print_confusion_matrix(metrics_rf.confusionMatrix(), title="Random Forest - Confusion Matrix:")
print_confusion_matrix(metrics_lr.confusionMatrix(), title="Logistic Regression - Confusion Matrix:")
print_confusion_matrix(metrics_dt.confusionMatrix(), title="Decision Tree - Confusion Matrix:")


**Decision Tree Focus**

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Configuration initiale du modèle
dt = DecisionTreeClassifier(featuresCol='selectedFeatures', labelCol='P_HABITABLE', maxBins=300)

# Grille de paramètres pour la recherche
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15]) \
    .addGrid(dt.minInstancesPerNode, [1, 2, 4]) \
    .build()

# Évaluateur
evaluator = MulticlassClassificationEvaluator(labelCol='P_HABITABLE', metricName="accuracy")

# Validation croisée
crossval = CrossValidator(estimator=dt,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

# Entraînement avec validation croisée
cvModel = crossval.fit(train_data)

# Meilleur modèle
bestModel = cvModel.bestModel
print(f"Meilleur modèle avec maxDepth: {bestModel._java_obj.getMaxDepth()} et minInstancesPerNode: {bestModel._java_obj.getMinInstancesPerNode()}")

# Prédictions et évaluation
predictions = bestModel.transform(test_data)
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy du meilleur modèle Decision Tree : {accuracy}")


In [ ]:
# Evaluateur pour les métriques supplémentaires
evaluator_f1 = MulticlassClassificationEvaluator(labelCol='P_HABITABLE', metricName="f1")
f1 = evaluator_f1.evaluate(predictions)
print(f"F1-Score du meilleur modèle Decision Tree : {f1}")

evaluator_precision = MulticlassClassificationEvaluator(labelCol='P_HABITABLE', metricName="weightedPrecision")
precision = evaluator_precision.evaluate(predictions)
print(f"Precision du meilleur modèle Decision Tree : {precision}")

evaluator_recall = MulticlassClassificationEvaluator(labelCol='P_HABITABLE', metricName="weightedRecall")
recall = evaluator_recall.evaluate(predictions)
print(f"Recall du meilleur modèle Decision Tree : {recall}")

In [ ]:
#save the model